In [1]:
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import pandas as pd

from segment_speed_utils.project_vars import (SEGMENT_GCS, 
                                              CONFIG_PATH, 
                                              PROJECT_CRS
                                             )
from segment_speed_utils import (helpers, wrangle_shapes, 
                                 segment_calcs)
from shared_utils import rt_dates
import test_split

analysis_date = rt_dates.DATES["jul2023"]

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from importlib import reload

STOP_SEG_DICT = helpers.get_parameters(CONFIG_PATH, "stop_segments")

In [3]:
gdf = test_split.put_all_together(analysis_date, STOP_SEG_DICT)

In [4]:
GROUPING_COL = STOP_SEG_DICT["grouping_col"]
SEGMENT_IDENTIFIER_COLS = STOP_SEG_DICT["segment_identifier_cols"]

In [5]:
part1 = gdf[gdf.n_vp_seg==1].reset_index(drop=True)
part2 = gdf[gdf.n_vp_seg==2].reset_index(drop=True)

In [8]:
part2_keep = (part2.groupby(["trip_instance_key"] + SEGMENT_IDENTIFIER_COLS)
              .vp_idx
              .max()
              .reset_index()
             )

part2_pared = pd.merge(
    part2,
    part2_keep,
    on = ["trip_instance_key", "vp_idx"] + SEGMENT_IDENTIFIER_COLS, 
    how = "inner"
)

In [10]:
part1_gdf = merge_in_segments(
    part1,
    SEGMENT_IDENTIFIER_COLS,
    GROUPING_COL
)

part2_gdf = merge_in_segments(
    part2_pared,
    SEGMENT_IDENTIFIER_COLS,
    GROUPING_COL
)

In [21]:
gdf2 = pd.concat(
    [part1_gdf, part2_gdf], 
    axis=0
).sort_values(
    SEGMENT_IDENTIFIER_COLS + ["trip_instance_key"]
).reset_index(drop=True)

In [23]:
gddf = dg.from_geopandas(gdf2, npartitions=50)

In [27]:
shape_meters_series = gddf.map_partitions(
    wrangle_shapes.project_point_geom_onto_linestring,
    "geometry",
    "vp_geometry",
    meta = ("shape_meters", "float")
)

In [30]:
gddf.columns

Index(['trip_instance_key', 'vp_idx', 'location_timestamp_local',
       'shape_array_key', 'stop_sequence', 'n_vp_seg', 'prior_vp_idx',
       'prior_location_timestamp_local', 'vp_geometry', 'prior_geometry',
       'geometry', 'shape_meters', 'location_timestamp_local_sec'],
      dtype='object')

In [32]:
shape_meters_series2 = gddf.map_partitions(
    wrangle_shapes.project_point_geom_onto_linestring,
    "geometry",
    "prior_geometry",
    meta = ("prior_shape_meters", "float")
)

In [28]:
TIMESTAMP_COL = "location_timestamp_local"

gddf["shape_meters"] = shape_meters_series
gddf["prior_shape_meters"] = shape_meters_series2

gddf = segment_calcs.convert_timestamp_to_seconds(
        gddf, [TIMESTAMP_COL, f"prior_{TIMESTAMP_COL}"])

In [37]:
gddf["distance"] = gddf.vp_geometry.distance(gddf.prior_geometry)

In [39]:
gdf3 = gddf.compute()

/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/opt/conda/lib/python3.9/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line,

In [41]:
gdf3 = gdf3.assign(
    meters_elapsed = abs(gdf3.shape_meters - gdf3.prior_shape_meters),
    sec_elapsed = abs(gdf3.location_timestamp_local_sec - gdf3.prior_location_timestamp_local_sec)
)

In [44]:
gdf3.columns

Index(['trip_instance_key', 'vp_idx', 'location_timestamp_local',
       'shape_array_key', 'stop_sequence', 'n_vp_seg', 'prior_vp_idx',
       'prior_location_timestamp_local', 'vp_geometry', 'prior_geometry',
       'geometry', 'shape_meters', 'location_timestamp_local_sec',
       'prior_shape_meters', 'prior_location_timestamp_local_sec', 'distance',
       'meters_elapsed', 'sec_elapsed'],
      dtype='object')

In [54]:
gdf3[gdf3.meters_elapsed > gdf3["distance"]*1.5][
    ["shape_meters", "prior_shape_meters",
     "meters_elapsed", "distance",
     "sec_elapsed"
    ]].shape

(51808, 5)

In [ ]:
SCALE = 1.5
gdf3 = gdf3.assign(
    meters_elapsed = gdf3.apply(
        lambda x: 
        x["distance"] if (
            x.meters_elapsed == 0 or 
            x.meters_elapsed >= x["distance"]*SCALE
        ) else x.meters_elapsed, 
        axis=1)
)

In [55]:
gdf3[gdf3.meters_elapsed <= gdf3["distance"]*1.5][
    ["shape_meters", "prior_shape_meters",
     "meters_elapsed", "distance",
     "sec_elapsed"
    ]]

,shape_meters,prior_shape_meters,meters_elapsed,distance,sec_elapsed
0,340.492168,294.216125,46.276043,36.792898,31
1,0.000000,0.000000,0.000000,55.583304,212
9,1828.303216,11.990035,1816.313182,1786.873286,121
10,1501.494667,0.000000,1501.494667,1494.377947,137
11,1828.303216,0.000000,1828.303216,1813.254069,152
...,...,...,...,...,...
2647856,267.609824,96.331438,171.278386,153.554969,411
2647857,247.932518,64.090787,183.841731,177.632313,409
2647858,300.479415,49.640185,250.839230,207.496161,32
2647859,32782.699204,32523.942558,258.756646,206.605579,27
